In [ ]:
!pip install pandas power-grid-model-ds[visualizer] --quiet

# ⚡ Advanced PGM-DS Workshop: Solving an Overload Scenario

You're a senior grid analyst at GridNova Utilities, responsible for operating a legacy distribution grid in a rural area. The grid is radially operated, with some cables inactive as back-up in case failures. Recently, customer load growth has increased dramatically, particularly in areas served by several long feeders. This has pushed some branches past their capacity, triggering repeated overloads.

Your task: upgrade the grid by adding a second substation and relieving the overloaded feeder through new connections to the new substation.

This hands-on simulation walks you through each step of diagnosing, planning, and solving this overload using the Power Grid Model Data Science library.

## 🎯 Workshop Goals
- Detect a line overload using PGM load flow calculations.
- Find a suitable node to create a connection to the new substation.
- Strategically open a line to reroute power and relieve the feeder.


In [ ]:
import numpy as np

from dataclasses import dataclass
from power_grid_model_ds import Grid, GraphContainer
from power_grid_model_ds.arrays import LineArray, NodeArray, SourceArray
from power_grid_model_ds.enums import NodeType
from power_grid_model_ds.visualizer import visualize

# 🧪 Step 1: Extend the Data Model
Goal: Add coordinate fields and tracking for simulated voltages and line currents. This allows us to store and analyse metadata of the grid needed to to decide where to invest in the grid.

You’ll subclass NodeArray and LineArray to add:

- x, y coordinates for spatial logic and plotting
- u for node voltage results
- i_from for line currents
- A computed .is_overloaded property for easy filtering

This shows how the Grid can be extended to suit the needs of a specific project.

In [ ]:
# 📦 Extend the grid with x, y, u (node) and i_from (line)
from numpy.typing import NDArray

class MyNodeArray(NodeArray):
    _defaults = {"x": 0.0, "y": 0.0, "u": 0.0}
    x: NDArray[np.float64]
    y: NDArray[np.float64]
    u: NDArray[np.float64]

class MyLineArray(LineArray):
    _defaults = {"i_from": 0.0, "overload_status": 0}
    i_from: NDArray[np.float64]
    overload_status: NDArray[np.int8]

    def set_overload_status(self):
        """Set the overload status based on the current and nominal current."""
        self.overload_status = np.where(self.i_from > self.i_n, 1, 0)

    @property
    def is_overloaded(self) -> NDArray[np.bool_]:
        """Check if the line is overloaded."""
        self.set_overload_status()
        return self.overload_status == 1

@dataclass
class MyGrid(Grid):
    node: MyNodeArray
    line: MyLineArray
    graphs: GraphContainer

# 🏗️ Step 2: Load and Prepare the Grid
Goal: Load a synthetic medium-voltage grid from the provided data
(Code is already given in helper.py file, take a look to see how the grid data is loaded!)

In [ ]:
from helper import load_dummy_grid

grid = load_dummy_grid(MyGrid)

We now loaded our the network visualised here

![input_network.png](input_network.png)

# 🧯 Step 3: Detect the Overload
This is your first excercise.

Goal: Identify which line(s) are exceeding their rated current (the `i_n` property).

You can do this step by step (don't forget to check the PGM-DS documentation):

1. Use the PowerGridModelInterface to calculate power flow
2. Update the Grid object with the calculated values
3. Return the lines (LineArray) that are overloaded

**💡 Hint**: You can use the `is_overloaded` property of the `MyLineArray` class to check for overloaded lines.

**💡 Hint**: https://power-grid-model-ds.readthedocs.io/en/stable/quick_start.html#performing-power-flow-calculations

In [ ]:
def check_for_capacity_issues(grid: Grid) -> LineArray:
    """Check for capacity issues on the grid.
    Return the lines that with capacity issues.
    """

print(check_for_capacity_issues(grid))

In [ ]:
# %load solutions/advanced_3_check_for_capacity_issues.py

We can use PGM-DSs visualization function to explore the resulting grid. Check out the highlighting parts of the grid based on it's attributes to find out where the overload occurs

In [ ]:
visualize(grid)

# 🧭 Step 4: Plan a Relief Strategy

If you visualize the grid and highlight the overloaded cables, this is what you will see:

![input_network_with_overload.png](input_network_with_overload.png)

We found out the north-east part of the area is overloaded.
Goal: Place a second substation near the overloaded path. In the next steps we will use this substation to relieve overloaded cables.

You’ll:
- Create a new substations using the NodeArrayobject at the correct location.

This substation will act as a new injection point for rerouting load.


In [ ]:
# Check the introduction workshop on adding a substation

def build_new_substation(grid: Grid, location: tuple[float, float]) -> NodeArray:
    """Build a new substation at the given location.
    Return the new substation.
    """

In [ ]:
# %load solutions/advanced_4_build_new_substation.py

# 🔗 Step 5: Analyze and Connect the Overloaded Route
Goal: Identify the best way to connect the new substation to the overloaded routes.

You’ll:
- Compute which routes (/feeders) are overloaded to see where we need to intervene.
- Find which node on an overloaded route is geographically closed to the new substation.
- Create a new cable to connect the closest node to the new substation.

**💡 Hint**: The lines have been extended with extra properties in Step 1

**💡 Hint**: The arrays in the grid have a filter option, https://power-grid-model-ds.readthedocs.io/en/stable/examples/model/array_examples.html#using-filters

In [ ]:
def get_all_congested_routes(grid: Grid) -> list[NodeArray]:
    """Get all nodes on routes that contain an overloaded line."""

In [ ]:
# %load solutions/advanced_5_1_get_all_congested_routes.py

Next we will use the nodes x and y coordinates to find a suitable node to connect to the new substation. You will create a find_connection_point function that return the Node in a route which is closest to the new_substation.

In [ ]:
def find_connection_point(route: NodeArray, new_substation: NodeArray) -> NodeArray:
    """Calculate the connection point for the new route.
    This should be the geographically closest node to the new substation.
    """
    # Calculate the distance of each node in the route to the new_substation
    # Return the closest one

In [ ]:
# %load solutions/advanced_5_2_find_connection_point.py

Finally we build a function that creates a new line between the connection point and the new substation.

❗ **IMPORTANT** ❗ The new line should first be created with an open connection; we will optimize the location of the line opening in the next step.

**💡 Hint**: In the introduction you learned how to add a LineArray to the grid.

In [ ]:
def connect_to_route(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Connect the new substation node to the connection point.
    """

In [ ]:
# %load solutions/advanced_5_3_connect_to_route.py

# 🔌 Step 6: Open the Right Line
Goal: Find the optimal line to open to relieve the original overloaded feeder.

You’ll:
- Trace a path from the newly created cable to the old substation
- Evaluate each line on the path by running `check_for_capacity_issues()` and find the optimal line to open
- Open the correct line
- Confirm the overload is resolved

This final step demonstrates how network topology can be programmatically optimized using the Power Grid Model Data Science toolkit!



In [ ]:
def optimize_route_transfer(grid: Grid, connection_point: NodeArray, new_substation: NodeArray) -> None:
    """Attempt to optimize the route transfer moving the normally open point (NOP) upstream towards the old substation.
    This way, the new substation will take over more nodes of the original route.
    """
    # Get the path from the connection point to the old substation
    ...

    # filter the first branch in the path
    ...

    # Iterate over the path and check if the route is still overloaded
    for from_node, to_node in zip(path[0:-1], path[1:]):
        # Check if the route is still overloaded
        ...
        
        # Move the Open Point (NOP) upstream
        ...
    
    grid.set_feeder_ids()

In [ ]:
# %load solutions/advanced_6_optimize_route_transfer.py

Now we combine the functions you created to solve the issues in the network

In [ ]:
def transfer_routes(grid: Grid, new_substation: NodeArray) -> NodeArray:
    """Migrate a subset of the routes of the old substation to the new substation.
    Each route can be migrated fully or partially.

    """
    congested_routes = get_all_congested_routes(grid)

    for route in congested_routes:
        closest_node = find_connection_point(
            route=route,
            new_substation=new_substation
        )

        connect_to_route(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation,
        )

        optimize_route_transfer(
            grid=grid,
            connection_point=closest_node,
            new_substation=new_substation)
        
        print(f"Connected new substation to node {closest_node.id}")

transfer_routes(grid=grid, new_substation=new_substation)

Check we resolved all contingencies

In [ ]:
print(check_for_capacity_issues(grid))

In [ ]:
visualize(grid)  

*Note: Jupyter notebook only supports one visualizer instance at a time. You might need to restart the kernel and re-run some cells for this final visualizer to work properly. If you do, make sure to not run earlier cells that contain `visualize(grid)`*

# ✅ Wrap-Up
You’ve just:

- Loaded a grid topology and grid loads from a file
- Analyse grid components that are or will soon be overloaded using load flow analysis
- Automatically optimize a solution to relieve (future) congestions on the energy grid

We hope you enjoyed working with Power Grid Model DS and would love to hear your feedback